In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import bokeh
import matplotlib.pyplot as plt
import seaborn as sns
import time


from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
import scipy.special


In [2]:
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [3]:
pd.set_option("display.max_columns", 1000)


In [4]:
log = pd.read_csv("./data/log.tsv", sep="\t")
events = pd.read_csv("./data/events.tsv", sep="\t")
users = pd.read_csv("./data/users.tsv", sep="\t")


In [5]:
log.loc[:,"user_id_str"] = log["user_id"].apply(lambda x:str(x))
log.loc[:,"event_id_str"] = log["event_id"].apply(lambda x:str(x))
log.head()

,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,user_id_str,event_id_str
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0,1,6261
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,1,127600
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,1,127600
3,1,125296,2017-06-06 14:01:27,1,NaN,NaN,NaN,1,125296
4,1,125128,2017-06-06 14:03:57,1,NaN,NaN,NaN,1,125128


In [6]:
users_interactions_count_df = log.groupby(["user_id", "event_id"]).size().groupby("user_id").size()
print(users_interactions_count_df.shape)
users_interactions_count_df.head()

(69773,)


user_id
1    498
2      5
3      2
4      3
5      7
dtype: int64

In [7]:
#ユーザーで層化してtrain_test_splitするには各人最低２回記録されてないといけない
#tutorialにしたがって5回にしておく

users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5 ].reset_index()[["user_id"]]
print(users_with_enough_interactions_df.shape)
display(users_with_enough_interactions_df.head())

(40654, 1)


,user_id
0,1
1,2
2,5
3,6
4,9


In [8]:
interactions_from_selected_users_df = log.merge(users_with_enough_interactions_df, how ="right", left_on = "user_id", right_on = "user_id")
print(interactions_from_selected_users_df.shape)
display(interactions_from_selected_users_df.head())

(1341063, 9)


,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,user_id_str,event_id_str
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0,1,6261
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,1,127600
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,1,127600
3,1,125296,2017-06-06 14:01:27,1,NaN,NaN,NaN,1,125296
4,1,125128,2017-06-06 14:03:57,1,NaN,NaN,NaN,1,125128


In [9]:
#複数回action_typeが記録された時に"smooth"にするらしい（よくわからない）
#def smooth_user_preference(x):
#    return math.log(1+x, 2)

#interactions_full_df = interactions_from_selected_users_df.groupby(["user_id","event_id"])["action_type"].sum().apply(smooth_user_preference).reset_index()
#print(interactions_full_df.shape)
#display(interactions_full_df.head())

In [10]:
interactions_full_df = interactions_from_selected_users_df.groupby(["user_id","event_id"])["action_type"].max().reset_index()
print(interactions_full_df.shape)
display(interactions_full_df.head())

(903244, 3)


,user_id,event_id,action_type
0,1,6261,3
1,1,7065,1
2,1,10308,3
3,1,14062,3
4,1,14255,3


In [11]:
#ユーザーで層化
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df, stratify = interactions_full_df["user_id"],
                                                              test_size = 0.20, random_state=42)
print(interactions_train_df.shape)
print(interactions_test_df.shape)

(722595, 3)
(180649, 3)


In [12]:
#user_idをindexにする
interactions_full_indexed_df = interactions_full_df.set_index('user_id')
interactions_train_indexed_df = interactions_train_df.set_index('user_id')
interactions_test_indexed_df = interactions_test_df.set_index('user_id')
interactions_test_indexed_df.head()

,event_id,action_type
user_id,,
22912,108389,2
54088,126760,2
20536,50599,2
52351,140614,1
45266,124985,1


In [13]:


def get_items_interacted(person_id, interactions_df):
    interacted_items = interactions_df.loc[person_id]["event_id"]
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [14]:
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100
#testデータから取ってくるデータの数


class ModelEvaluator:
    
    def get_not_interacted_items_sample(self, person_id, sample_size, seed =42):
        #インタラクトしてないアイテムからsample_size個無作為に返す
        interacted_items = get_items_interacted(person_id, interactions_full_indexed_df)
        all_items = set(log["event_id"])
        non_interacted_items = all_items - interacted_items
        
        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)
    
    def _verify_hit_top_n(self, item_id, recommended_items, topn):
        try:
            index = next(i for i,c in enumerate(recommended_items) if c == item_id)
        except:
            index = -1
        hit = int(index in range(0, topn))
        return hit, index
    
    def evaluate_model_for_user(self, model, person_id):
        
        #testからEVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMSだけとってきて，その中でランクの答え合わせをする（いいのか？）
        
        interacted_values_testset = interactions_test_indexed_df.loc[person_id]
        
        
        if type(interacted_values_testset["event_id"]) == pd.Series:
            
            person_interacted_items_testset = set(interacted_values_testset["event_id"])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset["event_id"])])
            
            
        interacted_items_count_testset = len(person_interacted_items_testset)
        
        person_recs_df = model.recommend_items(person_id, items_to_ignore = get_items_interacted(person_id, interactions_train_indexed_df),
                                              topn = 1000)
        
        hits_at_5_count = 0
        hits_at_10_count = 0
        
        for item_id in person_interacted_items_testset:
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id, 
                                        sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, seed = item_id%(2**32))
            
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))
            
         

            
            #print(person_recs_df[person_recs_df["event_id"].isin(items_to_filter_recs)])
            
            
            valid_recs_df = person_recs_df[person_recs_df["event_id"].isin(items_to_filter_recs)]
            #推薦した上位topn個のなかからtestとitem_idのデータを取り出す（有効な推薦）
            
            valid_recs = valid_recs_df["event_id"].values
            
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            #有効な推薦の上位5位のなかにitem_idがあるかどうか．あったら1がかえる
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10
            
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)
            
        person_metrics = {"hit@5_count": hits_at_5_count,
                             "hit@10_count": hits_at_10_count,
                             "interacted_count": interacted_items_count_testset,
                             "recall@5": recall_at_5,
                             "recall@10": recall_at_10}
            
        return person_metrics
    
    
    
    
        
    def evaluate_model(self, model):
        print ("Ruuning evaluation for users")
            
        people_metrics = []
        for idx, person_id in enumerate(list(interactions_test_indexed_df.index.unique().values)):
            if idx % 100 == 0 and idx > 0:
                print("%d userd processed" % idx)
            person_metrics = self.evaluate_model_for_user(model, person_id)
            person_metrics["_person_id"] = person_id
            people_metrics.append(person_metrics)
        print("%d users processed" % idx)
            
        detailed_results_df = pd.DataFrame(people_metrics).sort_values("interacted_count", ascending=False)
            
        global_recall_at_5 = detailed_results_df["hit@5_count"].sum()/float(detailed_results_df["interacted_count"].sum())
        global_recall_at_10 = detailed_results_df["hit@10_count"].sum()/float(detailed_results_df["interacted_count"].sum())
            
        global_metrics = {"modelName": model.get_model_name(),
                             "recall@5": global_recall_at_5,
                              "recall@10": global_recall_at_10
                             }
        return global_metrics, detailed_results_df
        
model_evaluator = ModelEvaluator()
            

# popularity推薦

In [15]:
#人気のeventを推薦
item_popularity_df = interactions_full_df.groupby("event_id")["action_type"].sum().sort_values(ascending=False).reset_index()
item_popularity_df.head(10)

,event_id,action_type
0,156106,310
1,133112,270
2,102445,262
3,137744,257
4,148883,255
5,133276,253
6,145557,248
7,111562,245
8,148641,237
9,122527,234


In [16]:
class PopularityRecommender:
    
    MODEL_NAME = "Popularity"
    
    def __init__(self, popularity_df, items_df=None):
        self.popularity_df = popularity_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def recommend_items(self, user_id, items_to_ignore=[], topn=20, verbose=False):
        recommendations_df = self.popularity_df[~self.popularity_df["event_id"].isin(items_to_ignore)].sort_values("action_type", ascending = False).head(topn)
        
        
        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')
                
            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'event_id', 
                                                          right_on = 'evnt_id')[['action_type', 'event_id']]
            
            
        return recommendations_df
        
popularity_model = PopularityRecommender(item_popularity_df, log)

In [17]:
#pop_global_metrics, pop_detailed_results_df = model_evaluator.evaluate_model(popularity_model)
#print("\nGlobal metrics:\n%s" % pop_global_metrics)
#pop_detailed_results_df.head(10)

# 推薦

In [18]:
#無視するイベント
ignore_event_list = []
events.loc[:,"event_date"] = pd.to_datetime(events["event_start_at"])
display(events.head())
ignore_event_list  =list(events.loc[events["event_date"] < pd.to_datetime("2017/9/23"),"event_id"])


In [20]:


recommend = popularity_model.recommend_items(user_id =1, items_to_ignore = ignore_event_list)
#recommend = popularity_model.recommend_items(user_id =1)
recommend.shape

(20, 2)

In [21]:
test = pd.read_csv("./data/test.tsv", sep="\t")
display(test.head())
print(test.shape)
user_list = test["user_id"]

,user_id
0,1
1,6
2,23
3,43
4,46


(2886, 1)


In [24]:
i = 0
recommends = []

for user in user_list:
    
    start = time.time() #時間計測
    i = i +1
    print("user_id:"+str(user))
    print("進捗:"+str(i*100/len(user_list))+"%") 
    
    result = popularity_model.recommend_items(user_id =user, items_to_ignore = ignore_event_list)
    #result = popularity_model.recommend_items(user_id =user)
    result = result.reset_index(drop = True)
    result.loc[:,"rank"] = result.index.values + 1
    result.loc[:,"user_id"] = user
    
    recommends.append(result)
    
    elapsed_time = time.time() - start #時間計測ここまで
    print ("予想残り時間")
    print(elapsed_time * (len(user_list)-i))

    

user_id:1
進捗:0.03465003465003465%
予想残り時間
31.24364733695984
user_id:6
進捗:0.0693000693000693%
予想残り時間
22.321532249450684
user_id:23
進捗:0.10395010395010396%
予想残り時間
22.692528247833252
user_id:43
進捗:0.1386001386001386%
予想残り時間
22.8956036567688
user_id:46
進捗:0.17325017325017325%
予想残り時間
23.50310730934143
user_id:48
進捗:0.2079002079002079%
予想残り時間
22.155303955078125
user_id:76
進捗:0.24255024255024255%
予想残り時間
22.719388246536255
user_id:79
進捗:0.2772002772002772%
予想残り時間
22.979102611541748
user_id:104
進捗:0.31185031185031187%
予想残り時間
23.209821939468384
user_id:139
進捗:0.3465003465003465%
予想残り時間
22.16498851776123
user_id:152
進捗:0.38115038115038113%
予想残り時間
22.508233785629272
user_id:198
進捗:0.4158004158004158%
予想残り時間
22.538776874542236
user_id:206
進捗:0.45045045045045046%
予想残り時間
22.576828002929688
user_id:233
進捗:0.4851004851004851%
予想残り時間
23.121553421020508
user_id:242
進捗:0.5197505197505198%
予想残り時間
22.724706888198853
user_id:247
進捗:0.5544005544005544%
予想残り時間
22.25080966949463
user_id:263
進捗:0.589050589050589%

予想残り時間
24.953600883483887
user_id:3063
進捗:5.301455301455301%
予想残り時間
21.47145652770996
user_id:3068
進捗:5.336105336105336%
予想残り時間
21.380226135253906
user_id:3076
進捗:5.37075537075537%
予想残り時間
21.467463970184326
user_id:3104
進捗:5.405405405405405%
予想残り時間
21.408183574676514
user_id:3117
進捗:5.44005544005544%
予想残り時間
23.501922845840454
user_id:3136
進捗:5.474705474705475%
予想残り時間
21.7645320892334
user_id:3163
進捗:5.509355509355509%
予想残り時間
21.341096878051758
user_id:3164
進捗:5.544005544005544%
予想残り時間
21.50030279159546
user_id:3191
進捗:5.578655578655579%
予想残り時間
21.235138177871704
user_id:3229
進捗:5.613305613305613%
予想残り時間
21.859911918640137
user_id:3267
進捗:5.647955647955648%
予想残り時間
21.542861223220825
user_id:3279
進捗:5.682605682605683%
予想残り時間
21.24940061569214
user_id:3319
進捗:5.717255717255718%
予想残り時間
21.46735453605652
user_id:3338
進捗:5.751905751905752%
予想残り時間
21.229896545410156
user_id:3347
進捗:5.786555786555787%
予想残り時間
21.89239239692688
user_id:3350
進捗:5.8212058212058215%
予想残り時間
22.093003749847412
user_i

予想残り時間
24.777404069900513
user_id:5926
進捗:10.602910602910603%
予想残り時間
20.26820182800293
user_id:5932
進捗:10.637560637560638%
予想残り時間
20.575780153274536
user_id:5972
進捗:10.672210672210673%
予想残り時間
20.21745538711548
user_id:6036
進捗:10.706860706860708%
予想残り時間
20.18995213508606
user_id:6039
進捗:10.74151074151074%
予想残り時間
20.285297393798828
user_id:6057
進捗:10.776160776160776%
予想残り時間
20.1399028301239
user_id:6117
進捗:10.81081081081081%
予想残り時間
20.398422718048096
user_id:6121
進捗:10.845460845460845%
予想残り時間
22.509357690811157
user_id:6158
進捗:10.88011088011088%
予想残り時間
20.277713775634766
user_id:6178
進捗:10.914760914760915%
予想残り時間
20.003798961639404
user_id:6217
進捗:10.94941094941095%
予想残り時間
20.28890609741211
user_id:6253
進捗:10.984060984060983%
予想残り時間
20.248549222946167
user_id:6256
進捗:11.018711018711018%
予想残り時間
20.30189323425293
user_id:6323
進捗:11.053361053361053%
予想残り時間
20.076720237731934
user_id:6328
進捗:11.088011088011088%
予想残り時間
20.14231300354004
user_id:6345
進捗:11.122661122661123%
予想残り時間
20.0182700157

予想残り時間
22.86750888824463
user_id:8935
進捗:15.904365904365905%
予想残り時間
19.059306621551514
user_id:8938
進捗:15.939015939015938%
予想残り時間
19.36841869354248
user_id:8974
進捗:15.973665973665973%
予想残り時間
19.10662055015564
user_id:8991
進捗:16.008316008316008%
予想残り時間
19.1224365234375
user_id:8994
進捗:16.042966042966043%
予想残り時間
18.86614179611206
user_id:8995
進捗:16.077616077616078%
予想残り時間
18.94959259033203
user_id:9021
進捗:16.112266112266113%
予想残り時間
19.036431312561035
user_id:9042
進捗:16.146916146916148%
予想残り時間
18.824896812438965
user_id:9044
進捗:16.181566181566183%
予想残り時間
18.972259521484375
user_id:9047
進捗:16.216216216216218%
予想残り時間
19.03590202331543
user_id:9057
進捗:16.250866250866252%
予想残り時間
20.798293113708496
user_id:9066
進捗:16.285516285516284%
予想残り時間
19.191234588623047
user_id:9094
進捗:16.32016632016632%
予想残り時間
19.134925603866577
user_id:9106
進捗:16.354816354816354%
予想残り時間
18.910022735595703
user_id:9115
進捗:16.38946638946639%
予想残り時間
19.011497020721436
user_id:9135
進捗:16.424116424116423%
予想残り時間
19.59248542

予想残り時間
21.456891298294067
user_id:11682
進捗:21.205821205821206%
予想残り時間
18.309415340423584
user_id:11684
進捗:21.24047124047124%
予想残り時間
18.00764012336731
user_id:11708
進捗:21.275121275121275%
予想残り時間
17.92821502685547
user_id:11713
進捗:21.30977130977131%
予想残り時間
18.08005142211914
user_id:11760
進捗:21.344421344421345%
予想残り時間
17.848029136657715
user_id:11832
進捗:21.37907137907138%
予想残り時間
17.773627042770386
user_id:11837
進捗:21.413721413721415%
予想残り時間
17.812837600708008
user_id:11857
進捗:21.44837144837145%
予想残り時間
18.002264261245728
user_id:11875
進捗:21.48302148302148%
予想残り時間
17.780381679534912
user_id:11881
進捗:21.517671517671516%
予想残り時間
17.89781928062439
user_id:11898
進捗:21.55232155232155%
予想残り時間
17.669687271118164
user_id:11933
進捗:21.586971586971586%
予想残り時間
17.841010332107544
user_id:11944
進捗:21.62162162162162%
予想残り時間
17.84768772125244
user_id:11960
進捗:21.656271656271656%
予想残り時間
19.460764169692993
user_id:11961
進捗:21.69092169092169%
予想残り時間
17.879862785339355
user_id:11973
進捗:21.725571725571726%
予想残り時

...

予想残り時間
7.0384275913238525
user_id:61575
進捗:73.73527373527374%
予想残り時間
5.98422384262085
user_id:61587
進捗:73.76992376992376%
予想残り時間
5.921642780303955
user_id:61639
進捗:73.8045738045738%
予想残り時間
5.8975982666015625
user_id:61668
進捗:73.83922383922383%
予想残り時間
5.837595462799072
user_id:61671
進捗:73.87387387387388%
予想残り時間
5.927117824554443
user_id:61755
進捗:73.9085239085239%
予想残り時間
5.984964609146118
user_id:61771
進捗:73.94317394317395%
予想残り時間
5.9273529052734375
user_id:61774
進捗:73.97782397782397%
予想残り時間
5.911771535873413
user_id:61777
進捗:74.01247401247402%
予想残り時間
5.972385406494141
user_id:61810
進捗:74.04712404712404%
予想残り時間
6.054781436920166
user_id:61866
進捗:74.08177408177409%
予想残り時間
5.901887893676758
user_id:61931
進捗:74.11642411642411%
予想残り時間
6.18180513381958
user_id:62018
進捗:74.15107415107416%
予想残り時間
5.859784126281738
user_id:62047
進捗:74.18572418572418%
予想残り時間
5.8068132400512695
user_id:62069
進捗:74.22037422037423%
予想残り時間
5.993608474731445
user_id:62072
進捗:74.25502425502425%
予想残り時間
5.870054006576538

予想残り時間
5.141036033630371
user_id:65792
進捗:79.002079002079%
予想残り時間
5.808740615844727
user_id:65829
進捗:79.03672903672904%
予想残り時間
4.855371713638306
user_id:65844
進捗:79.07137907137907%
予想残り時間
4.6841888427734375
user_id:65871
進捗:79.10602910602911%
予想残り時間
4.705043077468872
user_id:65875
進捗:79.14067914067914%
予想残り時間
4.8241190910339355
user_id:65879
進捗:79.17532917532918%
予想残り時間
4.7302751541137695
user_id:65885
進捗:79.20997920997921%
予想残り時間
4.787492752075195
user_id:65892
進捗:79.24462924462925%
予想残り時間
4.746523857116699
user_id:65947
進捗:79.27927927927928%
予想残り時間
4.734750270843506
user_id:65981
進捗:79.31392931392931%
予想残り時間
4.856215953826904
user_id:65982
進捗:79.34857934857935%
予想残り時間
4.795363426208496
user_id:65989
進捗:79.38322938322938%
予想残り時間
4.794268608093262
user_id:66010
進捗:79.41787941787942%
予想残り時間
4.79952335357666
user_id:66032
進捗:79.45252945252945%
予想残り時間
4.993761301040649
user_id:66034
進捗:79.48717948717949%
予想残り時間
4.786327362060547
user_id:66057
進捗:79.52182952182952%
予想残り時間
4.687358379364014

予想残り時間
3.6999878883361816
user_id:68485
進捗:84.33818433818433%
予想残り時間
4.323647499084473
user_id:68492
進捗:84.37283437283438%
予想残り時間
3.5337600708007812
user_id:68505
進捗:84.4074844074844%
予想残り時間
3.7069201469421387
user_id:68507
進捗:84.44213444213445%
予想残り時間
3.6490113735198975
user_id:68516
進捗:84.47678447678447%
予想残り時間
3.513031005859375
user_id:68571
進捗:84.51143451143452%
予想残り時間
3.604302406311035
user_id:68614
進捗:84.54608454608454%
予想残り時間
3.553173542022705
user_id:68632
進捗:84.58073458073459%
予想残り時間
3.5236692428588867
user_id:68677
進捗:84.61538461538461%
予想残り時間
3.479970932006836
user_id:68703
進捗:84.65003465003466%
予想残り時間
3.462944269180298
user_id:68714
進捗:84.68468468468468%
予想残り時間
3.4466967582702637
user_id:68728
進捗:84.71933471933473%
予想残り時間
3.445838212966919
user_id:68740
進捗:84.75398475398475%
予想残り時間
3.672800064086914
user_id:68745
進捗:84.7886347886348%
予想残り時間
3.472181797027588
user_id:68779
進捗:84.82328482328482%
予想残り時間
3.514188766479492
user_id:68798
進捗:84.85793485793486%
予想残り時間
3.58701610565

予想残り時間
3.2008893489837646
user_id:69700
進捗:89.60498960498961%
予想残り時間
2.3644208908081055
user_id:69701
進捗:89.63963963963964%
予想残り時間
2.379636526107788
user_id:69702
進捗:89.67428967428967%
予想残り時間
2.343045234680176
user_id:69703
進捗:89.70893970893971%
予想残り時間
2.3356783390045166
user_id:69704
進捗:89.74358974358974%
予想残り時間
2.3629589080810547
user_id:69705
進捗:89.77823977823978%
予想残り時間
2.3233962059020996
user_id:69706
進捗:89.81288981288981%
予想残り時間
2.3635354042053223
user_id:69707
進捗:89.84753984753985%
予想残り時間
2.3068060874938965
user_id:69708
進捗:89.88218988218988%
予想残り時間
2.284939765930176
user_id:69709
進捗:89.91683991683992%
予想残り時間
2.2909905910491943
user_id:69710
進捗:89.95148995148995%
予想残り時間
2.291276454925537
user_id:69711
進捗:89.98613998613999%
予想残り時間
2.285098075866699
user_id:69712
進捗:90.02079002079002%
予想残り時間
2.2798690795898438
user_id:69713
進捗:90.05544005544006%
予想残り時間
2.273526668548584
user_id:69714
進捗:90.09009009009009%
予想残り時間
2.305631160736084
user_id:69715
進捗:90.12474012474013%
予想残り時間
2.249937

予想残り時間
1.3510982990264893
user_id:69852
進捗:94.87179487179488%
予想残り時間
1.185502052307129
user_id:69853
進捗:94.9064449064449%
予想残り時間
1.170832872390747
user_id:69854
進捗:94.94109494109495%
予想残り時間
1.1555581092834473
user_id:69855
進捗:94.97574497574497%
予想残り時間
1.1302196979522705
user_id:69856
進捗:95.01039501039502%
予想残り時間
1.1402091979980469
user_id:69857
進捗:95.04504504504504%
予想残り時間
1.124415397644043
user_id:69858
進捗:95.07969507969509%
予想残り時間
1.1450586318969727
user_id:69859
進捗:95.11434511434511%
予想残り時間
1.1000161170959473
user_id:69860
進捗:95.14899514899516%
予想残り時間
1.1022615432739258
user_id:69861
進捗:95.18364518364518%
予想残り時間
1.1082077026367188
user_id:69862
進捗:95.21829521829522%
予想残り時間
1.1167845726013184
user_id:69863
進捗:95.25294525294525%
予想残り時間
1.0955939292907715
user_id:69864
進捗:95.28759528759528%
予想残り時間
1.0782585144042969
user_id:69865
進捗:95.32224532224532%
予想残り時間
1.0627985000610352
user_id:69866
進捗:95.35689535689535%
予想残り時間
1.045565128326416
user_id:69867
進捗:95.39154539154539%
予想残り時間
1.0955

In [25]:
recommends[1]

,event_id,action_type,rank,user_id
0,156106,310,1,6
1,133112,270,2,6
2,102445,262,3,6
3,137744,257,4,6
4,148883,255,5,6
5,133276,253,6,6
6,145557,248,7,6
7,111562,245,8,6
8,148641,237,9,6
9,122527,234,10,6


In [26]:
recommends[2]

,event_id,action_type,rank,user_id
0,156106,310,1,23
1,133112,270,2,23
2,102445,262,3,23
3,137744,257,4,23
4,148883,255,5,23
5,133276,253,6,23
6,145557,248,7,23
7,111562,245,8,23
8,148641,237,9,23
9,122527,234,10,23


In [27]:
f = open('recommend_lightGBM.txt', 'w')
for x in recommends:
    f.write(str(x) + "\n")
f.close()

In [28]:
sub = pd.DataFrame()

for recommend in recommends:
    
    sub = pd.concat([sub,recommend])

In [29]:
sub = sub.loc[:,["user_id","event_id","rank"]]
print(sub.shape)
sub.head(50)

(57720, 3)


,user_id,event_id,rank
0,1,156106,1
1,1,133112,2
2,1,102445,3
3,1,137744,4
4,1,148883,5
5,1,133276,6
6,1,145557,7
7,1,111562,8
8,1,148641,9
9,1,122527,10


In [30]:
sub.to_csv("sub_lightgbm.tsv",header=None,index=None,sep = "\t")